# Channel

在完成了对 `Cell` 的建模后，不难发现无论是 `SingleCompartment` 还是 `MultiCompartment` ，只要是 `HHTypedNeuron` ，在实际应用中都需要对离子通道进行设置。因此，对 `Channel` 进行建模也是十分重要的。

我们根据通道功能的不同，将`Channel` 分为以下几类：
- `Calcium Channels`：钙离子通道
- `Potassium Channels`：钾离子通道
- `Sodium Channels`：钠离子通道
- `Potassium Calcium Channels`：钙依赖性钾通道
- `Hyperpolarization-Activated Channels`：超极化激活通道
- `Leakage Channels`：漏电通道

以上这些通道提供了丰富的功能，对于每一种通道类别，`braincell` 已经内置好了许多的具体通道。如果想了解更多的内置的通道，你可以通过查阅 [离子通道合集](https://braincell.readthedocs.io/latest/apis/braincell.channel.html) 来学习使用。

同时，`braincell` 支持添加自定义的通道，给 `Channel` 的建模提供了很强的灵活性。

`braincell` 的通道建模具备以下几个显著优势：
- 接口统一：无论是线性通道还是复杂 Markov 模型，均可统一表示和集成。
- 灵活性强：支持多种电流类型，同时支持用户自定义建模。
- 兼容 `brainstate` 的自动微分系统：每个状态变量都能被精确地自动求导，支持高效训练与优化。
- 可拓展性高：通过继承与组合机制，通道模型可以轻松构建子类，实现参数修改与拓展。

在详细介绍具体通道之前，我们先要对 `Channel` 本身的建模进行讲解。

## 建模实现

在生物神经元中，离子通道控制着钠、钾、钙等离子的通透性，是神经信号发生与传递的基础。不同类型的通道具有各自的动力学特性，并对膜电位的变化产生影响。

在 `braincell` 中，我们使用 `IonChannel` 类作为通道建模的基础类。所有具体的通道模型都应继承自该类。

`IonChannel` 提供了以下接口，用于支持仿真过程中的关键操作：
- `current`：计算通道产生的离子电流。
- `compute_derivative`：计算通道内部状态变量的导数。
- `init_state`：初始化通道的状态变量。
- `reset_state`：重置通道状态。
- `pre_integral` 和 `post_integral`：积分前后的状态更新。
- `update_state`：可选的状态更新接口，用于定制行为。

以上所有这些方法都能与 `brainstate` 的微分系统集成。

在此之后，我们又定义了一个直接继承自 `IonChannel` 的基类 `Channel` 来进行封装。
因为 `Channel` 本身直接继承自 `IonChannel` ，并未引入新的行为或接口，所以从功能上讲二者是等价的。
当然，在语义上，我们更建议你用 `Channel` 来开发实际通道模型。

在完成了对 `Channel` 这个基类的建模之后，是时候对不同的具体通道进行建模了。
从实际应用上说，我们希望 `Channel` 的子类应该实现特定的方法，以定义特定通道类型的行为。

# Calcium Channels

## 电生理特性

钙离子通道是一类高度重要的膜通道，负责调节钙离子在细胞膜上的跨膜流动。钙离子在神经系统中不仅参与膜电位的变化，还在以下过程中扮演着关键角色：
- 动作电位的产生与延续：某些神经元类型依赖钙通道触发或维持放电节律。
- 突触传递：钙离子内流触发神经递质释放。
- 细胞内信号转导：钙离子可调控蛋白激酶活性、基因表达、酶促反应等。
- 钙依赖通道调控：许多钾通道或混合通道也受到钙浓度的调节。

钙通道通常具有缓慢激活、钙依赖失活、以及高选择性等特点。它们常常与钙泵和缓冲蛋白共同作用，以维持钙稳态。

## 建模实现

在 `braincell` 中，钙通道的建模通过继承 `CalciumChannel` 来实现，而 `CalciumChannel` 本身又继承自 `Channel`，属于 `IonChannel` 的一部分。

因此，在实际应用中我们需要实现相应的接口，如 `current(V, Ca)`、`compute_derivative(V, Ca)`等。
其中，`Ca` 参数是一个 `IonInfo` 类型，包含通道局部的钙浓度信息。而关于 `IonInfo` 的信息，我们会在 `Ion` 部分进行详细介绍。

通过这些接口，我们可以构建具有复杂动力学特性的钙通道，比如电压和钙浓度双重调控、钙依赖失活、稳态激活门控等。

在了解了基本的建模后，我们来看一个实际例子： `ICaN_IS2008`。

这是由 Inoue & Strowbridge 在 2008 年提出的一个钙激活的非选择性阳离子通道模型。
`braincell` 对离子通道有一套固定的命名规则，格式为 `通道类型_文献标识`。`braincell` 中大部分通道都是采用这种形式命名的，这样可以保证各通道的便于查找。

回到模型本身，该模型的电流由下列公式描述：

$$
\begin{aligned}
I_{\text{CAN}} &= g_{\text{max}} \cdot M([\mathrm{Ca}^{2+}]_i) \cdot p \cdot (V - E) \\\\
M([\mathrm{Ca}^{2+}]_i) &= \frac{[\mathrm{Ca}^{2+}]_i}{0.2 + [\mathrm{Ca}^{2+}]_i} \\\\
\frac{dp}{dt} &= \frac{\phi \cdot (p_\infty - p)}{\tau_p} \\\\
p_\infty &= \frac{1}{1 + \exp\left(-\frac{V + 43}{5.2}\right)} \\\\
\tau_p &= \frac{2.7}{\exp\left(-\frac{V + 55}{15}\right) + \exp\left(\frac{V + 55}{15}\right)} + 1.6
\end{aligned}
$$

其中：
- $M$ 是钙依赖激活函数
- $p$ 是电压门控激活变量
- $\phi$ 是温度因子
- $E$ 是反转电位
- $g_{\text{max}}$ 是最大电导

以下是该模型的代码实现：

```python
class ICaN_IS2008(CalciumChannel):
    def __init__(
        self,
        size,
        E=10. * u.mV,
        g_max=1. * (u.mS / u.cm ** 2),
        phi=1.,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.E = brainstate.init.param(E, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)

    def init_state(self, V, Ca: IonInfo, batch_size: int = None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, Ca: IonInfo, batch_size=None):
        V = V.to_decimal(u.mV)
        self.p.value = 1.0 / (1 + u.math.exp(-(V + 43.) / 5.2))

    def compute_derivative(self, V, Ca: IonInfo):
        V = V.to_decimal(u.mV)
        p_inf = 1.0 / (1 + u.math.exp(-(V + 43.) / 5.2))
        tau_p = 2.7 / (u.math.exp(-(V + 55.) / 15.) + u.math.exp((V + 55.) / 15.)) + 1.6
        self.p.derivative = self.phi * (p_inf - self.p.value) / tau_p / u.ms

    def current(self, V, Ca: IonInfo):
        M = Ca.C / (Ca.C + 0.2 * u.mM)
        g = self.g_max * M * self.p.value
        return g * (self.E - V)
```

在这个例子中，我们建立了一个具体的钙离子通道的子类，它继承自 `CalciumChannel` 。观察代码发现，在面对具体通道的建模时，`CalciumChannel` 具有很强的灵活性，只需要设定好相关的参数，并且依照公式将不同的接口用代码表达出来即可。

# Potassium Channels

## 电生理特性

钾离子通道是神经元中主要的超极化通道，决定了动作电位的恢复过程与静息膜电位的稳定性。其在神经元中的功能包括：
- 动作电位的复极：动作电位过程中，钠离子通道引起去极化，而钾通道开启导致钾外流，从而使膜电位回到静息水平。
- 控制兴奋性：某些钾通道具有延迟激活或钙依赖性质，能调节神经元的放电频率、适应性和持续兴奋性。
- 调控网络节律：有一些钾通道在节律放电、震荡中扮演重要角色。

钾通道功能广泛，其动力学建模在计算神经科学中具有重要地位。

## 建模实现

在 `braincell` 中，我们通过继承 `PotassiumChannel` 来实现钾通道的建模。该类继承自 `Channel`。

实现具体钾通道时，与实现钙通道相同，我们通常需要设定与钙通道相似的接口。

我们来看一个具体实例 `IKNI_Ya1989` ，便于更好的理解。

该模型最早由 Yamada 在 1989 年提出，是一类经典的缓慢非灭活钾离子通道，用于解释 AHP 期间的频率调节机制。

该模型由下列公式描述：

$$
\begin{aligned}
I_{M} &= \bar{g}_M \cdot p \cdot (V - E_K) \\\\
\frac{dp}{dt} &= \frac{p_{\infty}(V) - p}{\tau_p(V)} \\\\
p_{\infty}(V) &= \frac{1}{1 + \exp\left(-\frac{V - V_{\text{sh}} + 35}{10}\right)} \\\\
\tau_p(V) &= \frac{\tau_{\max}}{3.3 \exp\left(\frac{V - V_{\text{sh}} + 35}{20}\right) + \exp\left(-\frac{V - V_{\text{sh}} + 35}{20}\right)}
\end{aligned}
$$

其中：
- $V$ 是膜电位
- $E_K$ 是钾离子反转电位
- $\bar{g}_M$ 是最大电导
- $p$ 是门控变量
- $V_{\text{sh}}$ 是膜电位偏移
- $\tau_{\max}$ 是最大时间常数
- $p_{\infty}$ 是稳态激活函数
- $\tau_p$ 是激活时间常数

以下是代码实现：

```python
class IKNI_Ya1989(PotassiumChannel):
    def __init__(
        self,
        size,
        g_max=0.004 * (u.mS * u.cm ** -2),
        phi_p=1.,
        phi_q=1.,
        tau_max=4e3 * u.ms,
        V_sh=0. * u.mV,
        name=None,
    ):
        super().__init__(size=size, name=name)

        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.tau_max = brainstate.init.param(tau_max, self.varshape, allow_none=False)
        self.V_sh = brainstate.init.param(V_sh, self.varshape, allow_none=False)
        self.phi_p = brainstate.init.param(phi_p, self.varshape, allow_none=False)
        self.phi_q = brainstate.init.param(phi_q, self.varshape, allow_none=False)

    def init_state(self, V, Ca: IonInfo, batch_size: int = None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, K: IonInfo, batch_size=None):
        self.p.value = self.f_p_inf(V)
        if isinstance(batch_size, int):
            assert self.p.value.shape[0] == batch_size

    def compute_derivative(self, V, K: IonInfo):
        self.p.derivative = self.phi_p * (self.f_p_inf(V) - self.p.value) / self.f_p_tau(V)

    def current(self, V, K: IonInfo):
        return self.g_max * self.p.value * (K.E - V)

    def f_p_inf(self, V):
        V = (V - self.V_sh).to_decimal(u.mV)
        return 1. / (1. + u.math.exp(-(V + 35.) / 10.))

    def f_p_tau(self, V):
        V = (V - self.V_sh).to_decimal(u.mV)
        temp = V + 35.
        return self.tau_max / (3.3 * u.math.exp(temp / 20.) + u.math.exp(-temp / 20.))
```

从这个例子可以看出，`PotassiumChannel` 提供了统一的接口，而 `IKNI_Ya1989` 可以通过简单的数学表达式，轻松构建一个具备具体生物意义的离子通道模型。

在实际应用中，我们还可以在子类下进一步构建子类。

比如，先构建 `IKK2_pq_ss` ,其继承自 `PotassiumChannel` 。而在后续工作中，我们可以构建继承自  `IKK2_pq_ss` 的 `KK2A_HM1992` ，并且在 `KK2A_HM1992` 中对原本的 `IKK2_pq_ss` 进行拓展。这样的流程方便大批量建模，提高工作效率。如果你对这一部分感兴趣，可以参考我们的 [离子通道合集](https://braincell.readthedocs.io/latest/apis/braincell.channel.html) ，观察 `braincell` 中是如何具体实现的。

# Sodium Channels

## 电生理特性

钠离子通道是神经元中不可或缺的兴奋性通道，它在动作电位的启动和上升相中发挥着核心作用。在膜电位达到一定阈值时，钠通道快速激活并造成钠离子内流，导致膜电位迅速去极化。

钠通道具有典型的双门控机制：
- 激活门：快速响应膜电位的变化，决定通道是否打开。
- 失活门：稍慢地响应膜电位，关闭通道使其失活。

这种双门控结构使钠通道呈现出快速激活与快速失活的动力学行为，是动作电位快速起落的根本机制。

钠通道对膜电位变化极为敏感，其动力学特性决定了其对神经元兴奋性的调节作用。在经典的 HH 模型中，钠电流通常表示为：

$$
I_{Na} = \bar{g}_{Na} \cdot m^3 h \cdot (V - E_{Na})
$$

其中：
- $m$ 为激活变量
- $h$ 为失活变量
- $g_{Na}$ 是最大电导
- $E_{Na}$ 是钠离子的反转电位

## 建模实现

在 `braincell` 中，我们通过继承 `SodiumChannel` 类来实现钠离子通道的建模。该类本质上也一样是 `Channel` 的子类。

与前面介绍的钙通道、钾通道一致，我们只需定义相应的接口就可以完成建模。

下面我们依旧来看一个具体的例子：`INa_p3q_markov`。

值得一提的是，这里我们采用了不同的通道命名方式。 `INa_p3q_markov` 是根据公式结构来命名的，表示电流为 $p^3 q$ 的形式，并采用 Markov 动力学形式。
这里是 `braincell` 对离子通道的另一套命名规则，即 `通道类别与名称_电流形式_模型` 。有部分离子通道采用这种命名规则。

`INa_p3q_markov` 是一个采用 Markov 链建模形式的钠通道模型，其中电流由激活变量 $p$ 和失活变量 $q$ 控制：

$$
\begin{aligned}
I_{\mathrm{Na}} &= g_{\mathrm{max}} \cdot p^3 \cdot q \cdot (V - E_{Na}) \\\\
\frac{dp}{dt} &= \phi \cdot (\alpha_p (1 - p) - \beta_p p) \\\\
\frac{dq}{dt} &= \phi \cdot (\alpha_q (1 - q) - \beta_q q)
\end{aligned}
$$

其中：
- `p` 是激活变量
- `q` 是失活变量
- `φ` 是温度因子
- $g_{max}$ 是最大电导
- $E_{Na}$ 是钠离子反转电位

具体代码如下：

```python
class INa_p3q_markov(SodiumChannel):
    def __init__(
        self,
        size,
        g_max=90. * (u.mS / u.cm ** 2),
        phi=1.,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)

    def init_state(self, V, Na: IonInfo, batch_size=None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))
        self.q = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, Na: IonInfo, batch_size=None):
        self.p.value = self.f_p_alpha(V) / (self.f_p_alpha(V) + self.f_p_beta(V))
        self.q.value = self.f_q_alpha(V) / (self.f_q_alpha(V) + self.f_q_beta(V))

    def compute_derivative(self, V, Na: IonInfo):
        self.p.derivative = self.phi * (self.f_p_alpha(V) * (1. - self.p.value) - self.f_p_beta(V) * self.p.value) / u.ms
        self.q.derivative = self.phi * (self.f_q_alpha(V) * (1. - self.q.value) - self.f_q_beta(V) * self.q.value) / u.ms

    def current(self, V, Na: IonInfo):
        return self.g_max * self.p.value ** 3 * self.q.value * (Na.E - V)

    def f_p_alpha(self, V):
        raise NotImplementedError

    def f_p_beta(self, V):
        raise NotImplementedError

    def f_q_alpha(self, V):
        raise NotImplementedError

    def f_q_beta(self, V):
        raise NotImplementedError
```

与其他两种离子通道完全相同，我们的钠通道建模依旧保持了高度灵活性。

# Potassium Calcium Channels

## 电生理特性

钙依赖性钾通道是一类在神经元中广泛存在的钾通道，其激活不仅依赖于膜电位，还依赖于胞内钙离子浓度的变化。

这类通道在神经元的兴奋-抑制调控、动作电位后超极化、放电频率调节等方面起着至关重要的作用。它们通常在钙离子内流后被激活，促进钾离子外流，从而使膜电位恢复或进入超极化状态。

## 建模实现

钙依赖性钾通道，在电生理学中经常简称为 `KCa` 通道，因此在建模时我们也沿用了这个写法，便于代码保持简洁性。

在 `braincell` 中，我们通过直接继承 `KCaChannel` 类来实现钠离子通道的建模。该类也是 `Channel` 的子类。

与前面介绍的通道一致，我们只需定义相应的接口就可以完成建模。

同样，我们还是给出一个例子 `IAHP_De1994` 。

`IAHP_De1994` 模型由 Destexhe 等人在 1994 年提出，其继承自 `KCaChannel` ，用于模拟慢性钙依赖钾通道的动力学行为，表达式如下：

$$
    \begin{aligned}
    (\text{closed}) + n \mathrm{Ca}_{i}^{2+} \underset{\beta}{\stackrel{\alpha}{\rightleftharpoons}} (\text{open})
    \end{aligned}
$$

$$
\begin{aligned}
        I_{AHP} &= g_{\mathrm{max}} p^2 (V - E_K) \\
        {dp \over dt} &= \phi {p_{\infty}(V, [Ca^{2+}]_i) - p \over \tau_p(V, [Ca^{2+}]_i)} \\
        p_{\infty} &=\frac{\alpha[Ca^{2+}]_i^n}{\left(\alpha[Ca^{2+}]_i^n + \beta\right)} \\
        \tau_p &=\frac{1}{\left(\alpha[Ca^{2+}]_i +\beta\right)}
        \end{aligned}
$$

其中：
- $p$ 是通道状态变量
- $\alpha, \beta$ 是通道的开启/关闭速率常数
- $n$ 是钙离子结合数
- $\phi$ 是温度因子
- $g_{\text{max}}$ 是最大电导
- $E_K$ 是钾离子反转电位

代码实现如下：

```python
class IAHP_De1994(KCaChannel):
    def __init__(
        self,
        size: brainstate.typing.Size,
        n: Union[brainstate.typing.ArrayLike, Callable] = 2,
        g_max: Union[brainstate.typing.ArrayLike, Callable] = 10. * (u.mS / u.cm ** 2),
        alpha: Union[brainstate.typing.ArrayLike, Callable] = 48.,
        beta: Union[brainstate.typing.ArrayLike, Callable] = 0.09,
        phi: Union[brainstate.typing.ArrayLike, Callable] = 1.,
        name: Optional[str] = None,
    ):
        super().__init__(size=size, name=name)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.n = brainstate.init.param(n, self.varshape, allow_none=False)
        self.alpha = brainstate.init.param(alpha, self.varshape, allow_none=False)
        self.beta = brainstate.init.param(beta, self.varshape, allow_none=False)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)

    def init_state(self, V, K: IonInfo, Ca: IonInfo, batch_size=None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, K: IonInfo, Ca: IonInfo, batch_size=None):
        C2 = self.alpha * u.math.power(Ca.C / u.mM, self.n)
        C3 = C2 + self.beta
        if batch_size is None:
            self.p.value = u.math.broadcast_to(C2 / C3, self.varshape)
        else:
            self.p.value = u.math.broadcast_to(C2 / C3, (batch_size,) + self.varshape)
            assert self.p.value.shape[0] == batch_size

    def compute_derivative(self, V, K: IonInfo, Ca: IonInfo):
        C2 = self.alpha * u.math.power(Ca.C / u.mM, self.n)
        C3 = C2 + self.beta
        self.p.derivative = self.phi * (C2 / C3 - self.p.value) * C3 / u.ms

    def current(self, V, K: IonInfo, Ca: IonInfo):
        return self.g_max * self.p.value * self.p.value * (K.E - V)
```

其实这几个例子的建模方式高度统一，在实现方式上没有区别。

# Hyperpolarization-Activated Channels

## 电生理特性

超极化激活通道，是一类在膜电位超极化时被激活的阳离子通道。它们在心脏节律神经元和中枢神经系统中的某些类型神经元中发挥着关键的调节作用：
- 调节静息膜电位：在静息状态下产生的去极化电流可稳定神经元的膜电位，防止过度超极化。
- 参与节律性放电：在丘脑等结构中，帮助神经元维持固有节律放电。
- 兴奋性调节：能够降低神经元的输入电阻，从而降低细胞对输入的敏感性。

## 建模实现

由于我们在 `braincell` 中没有内置很多的超极化激活通道，所以我们直接让具体的超极化激活通道继承自 `Channel` 类，这在实际使用上并不会有任何影响。

下面我们给出一个超极化激活通道的例子 `Ih_HM1992` 。

`Ih_HM1992` 是由 Huguenard & McCormick 在 1992 年提出的超极化激活阳离子电流模型，其动力学由下式给出：

$$
\begin{aligned}
        I_h &= g_{\mathrm{max}} p \\
        \frac{dp}{dt} &= \phi \frac{p_{\infty} - p}{\tau_p} \\
        p_{\infty} &=\frac{1}{1+\exp ((V+75) / 5.5)} \\
        \tau_{p} &=\frac{1}{\exp (-0.086 V-14.59)+\exp (0.0701 V-1.87)}
        \end{aligned}
        $$

其中：
- $p$ 是激活变量
- $g_{\text{max}}$ 是最大电导
- $E$ 是反转电位（约 43 mV）
- $\phi$ 是温度因子

具体代码实现如下：

```python
class Ih_HM1992(Channel):
    def __init__(
        self,
        size,
        g_max=10. * (u.mS / u.cm ** 2),
        E=43. * u.mV,
        phi=1.,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.E = brainstate.init.param(E, self.varshape, allow_none=False)

    def init_state(self, V, batch_size=None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, batch_size=None):
        self.p.value = self.f_p_inf(V)

    def compute_derivative(self, V):
        self.p.derivative = self.phi * (self.f_p_inf(V) - self.p.value) / self.f_p_tau(V) / u.ms

    def current(self, V):
        return self.g_max * self.p.value * (self.E - V)

    def f_p_inf(self, V):
        V = V.to_decimal(u.mV)
        return 1. / (1. + u.math.exp((V + 75.) / 5.5))

    def f_p_tau(self, V):
        V = V.to_decimal(u.mV)
        return 1. / (u.math.exp(-0.086 * V - 14.59) + u.math.exp(0.0701 * V - 1.87))
```

通过这个例子你可以看到，超极化激活通道的建模可以非常自然地融入 `braincell` 框架，借助 `Channel` 的接口，轻松表达其独特的动力学性质。

# Leakage Channels

## 电生理特性

漏电通道是一类始终开放的离子通道，其通透性不受膜电位、电压门控、配体结合等机制的调控。在电生理学中，漏电通道主要负责维持神经元的静息膜电位，通常被建模为一个线性电导。

在经典的 HH 模型中，漏电流通道主要用于模拟未明确建模的背景离子流，提供一个稳定的背景电流。

## 建模实现

在 `braincell` 中，漏电通道通过继承 `LeakageChannel` 实现。由于漏电电流无激活或失活门控变量，因此该类通道不需要实现 `compute_derivative` 或 `init_state` 等微分过程，相对比较简单。

我们来看一个具体的实现：`IL`，即最基础的线性漏电通道，其数学表达式为：

$$
I_L = g_L (E_L - V)
$$

其中：
- $g_L$ 为漏电通道电导（通常为常数）
- $E_L$ 为漏电反转电位
- $V$ 为膜电位

实现代码也相当简单：

```python
class IL(LeakageChannel):
    def __init__(
        self,
        size,
        g_max=0.1 * (u.mS / u.cm ** 2),
        E=-70. * u.mV,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.E = brainstate.init.param(E, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)

    def current(self, V):
        return self.g_max * (self.E - V)
```

这个 `IL` 类提供了一个最简单的通道模型，它不涉及任何内部状态或动力学，是一个静态电导模型，常作为神经元模型中的基础部分。







